In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


In [8]:
examples = [
    {"movie": "Top Gun.", "answer": """🛩️👨‍✈️🔥"""},
    {"movie": "The Godfather.","answer": """👨‍👨‍👦🔫🍝"""},
    {"movie": "Frozen.","answer": """❄️👭🎶"""},
    {"movie": "Parasite.","answer": """🏠🌧️🎭"""},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [9]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]



final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",  "You are a helpful AI giving the three emojis that represent the given movie best."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


In [10]:
def invoke_chain(movie):
    result = chain.invoke({"input": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [11]:
invoke_chain("Toy Story")


content='🤠🚀🎮'


In [12]:
invoke_chain("Dark Night")

content='🦇🤵🏻💥'
